Ident-O-Matic
===========

So, what we got...

In [1]:
import os
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import fileupload
import subprocess
import tempfile

In [10]:
#This is where the results go...
results = widgets.Output()

tmp_dir = tempfile.mkdtemp()
tmp_file = None

def clear_all(b):
    #select_csv.value =o ptions[0][1]
    csv_url.value = ''
    results.clear_output()

def analyse_csv(b):
    '''
    Try to open the CSV file, and start the analysis.
    '''
    # Hacky reliance on global here:
    global tab
    # This makes sure the results get used properly:
    results.clear_output()
    with results:
        if tab.selected_index == 2:
            output = subprocess.run(["sf", "-json", tmp_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE).stdout.decode('utf-8')
            display(HTML("<pre>%s</pre>" % output))

def _cb(change):
    # Hacky reliance on global here:
    global tmp_file
    filename = change['owner'].filename
    tmp_file = os.path.join(tmp_dir, filename)
    #print('Storing to %s' % tmp_file)
    with open(tmp_file,"wb") as f:
        f.write(change['owner'].data)
    _upload_label.value= 'Uploaded `{}` ({:.2f} kB)'.format(
        filename, len(change['owner'].data) / 2 **10)

_upload_widget = fileupload.FileUploadWidget()
_upload_widget.observe(_cb, names='data')
_upload_label = widgets.Label(value="")

upload_tab = widgets.VBox([_upload_widget, _upload_label])

select_csv = widgets.Dropdown(
        options=[],
        description='',
        disabled=False,
        layout=widgets.Layout(width='90%')
    )

csv_url = widgets.Text(
        placeholder='Enter the URL to fetch',
        description='URL:',
        disabled=False,
        layout=widgets.Layout(width='90%')
    )

clear_button = widgets.Button(
        description='Clear',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Clear current data',
        icon=''
    )

analyse_button = widgets.Button(
        description='Analyse',
        disabled=False,
        button_style='primary', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Analyse',
        icon=''
    )

clear_button.on_click(clear_all)
analyse_button.on_click(analyse_csv)
select_note = widgets.HTML('Select an example file to analyse:')
select_tab = widgets.VBox([select_note, select_csv])
tab = widgets.Tab(children=[select_tab, csv_url, upload_tab ])
tab.set_title(0, 'Select example')
tab.set_title(1, 'Fetch from URL')
tab.set_title(2, 'Upload')
display(widgets.VBox([tab, widgets.HBox([analyse_button, clear_button]), results]))

In [56]:
!file --mime /tmp/tmphk4vw8ix/datacite-1.xml

/tmp/tmphk4vw8ix/datacite-1.xml: text/xml; charset=us-ascii


In [57]:
!sf /tmp/tmphk4vw8ix/datacite-1.xml

---
siegfried   : 1.7.11
scandate    : 2019-06-15T21:32:11Z
signature   : default.sig
created     : 2019-02-16T11:09:29+01:00
identifiers : 
  - name    : 'pronom'
    details : 'DROID_SignatureFile_V94.xml; container-signature-20180917.xml'
---
filename : '/tmp/tmphk4vw8ix/datacite-1.xml'
filesize : 826
modified : 2019-06-15T21:30:45Z
errors   : 
matches  :
  - ns      : 'pronom'
    id      : 'fmt/101'
    format  : 'Extensible Markup Language'
    version : '1.0'
    mime    : 'application/xml'
    basis   : 'extension match xml; byte match at 0, 19'


In [ ]:
!

Created by [Andrew Jackson](https://anjackson.net/).

See https://www.digipres.net/guides/format-id/ for more information.